## Relaxation Methods --
    ### Solving 2D Laplace + Poisson

In [ ]:
import numpy as np
from matplotlib import pyplot

In [2]:
# Grid 
# Lx, Ly = 5, 5
nx, ny = 5, 5
delta = 0.25
f = 1/2
x = np.linspace(0, 1., num=nx)
y = np.linspace(0, 1., num=ny)
# Initialize initial guess
u_0 = np.zeros((ny, nx))

#Boundary Conditions
u_0[:, 0] = 1.0
u_0[0, :] = 1.0
u_0[ny-1, :] = 1.0
u_0[:,nx-1] = 0.0

In [3]:
# jacobi Method function
def jacobi_meth(u_0, delta, b, maxiter = 20000, rtol = 1e-6):
    u = u_0.copy()
    ny, nx = u.shape
    rdiff = rtol + 1
    ite = 0
    while rdiff > rtol and ite < maxiter:
        u_n = u.copy()
        for j in range(1, ny-1):
            for i in range(1, nx-1):
                u[j, i] = (1/4) * (u_n[j-1, i] + 
                                   u_n[j, i-1] + 
                                   u_n[j, i+1] + 
                                   u_n[j+1, i] - 
                                   b[j, i])
        
    return u

In [4]:
# Start Test Problem

In [5]:
# set parameters
nx, ny = 128, 128
Lx, Ly = 5, 5
dx = Lx / (nx-1)
dy = Ly / (ny-1)

x = np.linspace(0.0, Lx, nx)
y = np.linspace(0.0, Ly, ny)

# Initial Conditions set
p0 = np.zeros((ny, nx))
p0[-1, :] = np.sin(1.5 * np.pi * x / Lx)

In [6]:
def laplace_2d_jacobi(p0, maxiter = 20000, rtol=1e-6):
    # Solves the 2D laplace eqn on uniform grid using Jacobi method
    p = p0.copy()
    diff = rtol + 1.0
    ite = 0
    while diff > rtol and ite < maxiter:
        pn = p.copy()
        # Update interior points solution
        p[1:-1, 1:-1] = 0.25 * (pn[1:-1, :-2] + pn[1:-1, 2:] +
                                pn[:-2, 1:-1] + pn[2:, 1:-1])
        # Apply Neuman condition at R boundary (0.0)
        p[1:-1, -1] = 0.25 * (2.0 * pn[1:-1, -2] +
                              pn[2:, -1] + pn[:-2, -1])
        
        diff = l2_norm(p, pn)
        ite += 1
    return p, ite, diff

In [7]:
# L2-Norm Function
def l2_norm(u, u_ref):
    l2_diff = (np.sqrt(np.sum((u-u_ref)**2)) / 
              np.sqrt(np.sum(u_ref**2)))
    return l2_diff

In [8]:
# Plot_3D Function
def plot_3d(x, y, p, label='$z$', elev=30.0, azim=45.0):
    """Create a 3D surface plot of the scalar field `p`.

    Parameters
    ----------
    x : numpy.ndarray
        Gridline locations in the x direction as a 1D array of floats.
    y : numpy.ndarray
        Gridline locations in the y direction as a 1D array of floats.
    p : numpy.ndarray
        Scalar field to plot as a 2D array of floats.
    label : string, optional
        Axis label to use in the third direction; default: 'z'.
    elev : float, optional
        Elevation angle in the z plane; default: 30.0.
    azim : float, optional
        Azimuth angle in the x,y plane; default: 45.0.

    """
    fig = pyplot.figure(figsize=(8.0, 6.0))
    ax = mplot3d.Axes3D(fig)
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel(label)
    X, Y = numpy.meshgrid(x, y)
    ax.plot_surface(X, Y, p, cmap=cm.viridis)
    ax.set_xlim(x[0], x[-1])
    ax.set_ylim(y[0], y[-1])
    ax.view_init(elev=elev, azim=azim)

In [9]:
# Laplace Solution Function
def laplace_solution(x, y, Lx, Ly):
    """Return the analytical solution.

    Parameters
    ----------
    x : numpy.ndarray
        Coordinates along the x direction as a 1D array of floats.
    y : numpy.ndarray
        Coordinates along the y direction as a 1D array of floats.
    Lx : float
        Length of the domain in the x direction.
    Ly : float
        Length of the domain in the y direction.

    Returns
    -------
    u : numpy.ndarray
        The analytical solution as a 2D array of floats.

    """
    X, Y = np.meshgrid(x, y)
    u = (np.sinh(1.5 * np.pi * Y / Ly) /
         np.sinh(1.5 * np.pi * Ly / Lx) *
         np.sin(1.5 * np.pi * X / Lx))
    
    return u

In [10]:
# Solution using Jacobi relaxation method
p, ites, diff = laplace_2d_jacobi(p0, maxiter=20000, rtol=1e-8)
print('Jacobi relaxation: {} iterations '.format(ites) + 
      'to reach relative difference of {}'.format(diff))

Jacobi relaxation: 19993 iterations to reach relative difference of 9.998616841158966e-09


In [ ]:
%%timeit
laplace_2d_jacobi(p0, maxiter=20000, rtol=1e-8)

In [11]:
# Compute analytical solution
p_exact = laplace_solution(x, y, Lx, Ly)

# Compute relative L2 norm of error
l2_norm(p, p_exact)

6.173551335297434e-05

### Gauss-Seidel Method

In [12]:
def laplace_2d_gs(p0, maxiter=20000, rtol=1e-6):
    ny, nx = p0.shape
    p = p0.copy()
    diff = rtol + 1.0
    ite = 0
    while diff > rtol and ite < maxiter:
        pn = p.copy()
        # update interior
        for j in range(1, ny-1):
            for i in range(1, nx-1):
                p[j, i] = (1/4) * (p[j, i-1] + p[j, i+1] + 
                                   p[j-1, i] + p[j+1, i])
        # Apply zero-grad @ R-Boundary
        for j in range(1, ny-1):
            p[j,-1] = (1/4) * (2. * p[j, -2] + p[j-1, -1] + p[j+1, -1])
        
        diff = l2_norm(p,pn)
        ite+=1
    return p, ite, diff

In [ ]:
p, ites, diff = laplace_2d_gs(p0, maxiter=20000, rtol=1e-8)

In [ ]:
def poisson2d_sor(p0, f, w=0.2, maxiter=20000, rtol=1e-6):
    p=p0.copy()
    ny, nx = p.shape
    rdiff = rtol + 1
    ite = 0
    while rdiff > rtol and ite < maxiter:
        pn = p.copy()
        for j in range(1, ny - 1):
            for i in range(1, nx - 1):
                p[j, i] = ((1-w) * pn[j,i] + 
                          (1/4) * w * (p[j-1,i] + 
                                       p[j, i-1] + 
                                       pn[j,i+1] + 
                                       pn[j+1, i] - f))
    return p

## NUMBAH

In [ ]:
conda install numba

In [ ]:
import numba
from numba import jit

In [ ]:
@jit(nopython=True)
def fib_it(n):
    a, b = 1, 1
    for i in range(n-2):
        a,b = b, a+b
    return b

In [ ]:
%%timeit
fib_it(500000)

In [ ]:
print(numba.__version__)

In [ ]:
@jit(nopython=True)
def laplace_2d_jacobi(p0, maxiter=20000, rtol=1e-6):
    """
    Solves the 2D Laplace equation on a uniform grid
    with equal grid spacing in both directions
    using Jacobi relaxation method.
    
    The exit criterion of the solver is based on the relative L2-norm
    of the solution difference between two consecutive iterations.
    
    Parameters
    ----------
    p0 : numpy.ndarray
        The initial solution as a 2D array of floats.
    maxiter : integer, optional
        Maximum number of iterations to perform;
        default: 20000.
    rtol : float, optional
        Relative tolerance for convergence;
        default: 1e-6.
    
    Returns
    -------
    p : numpy.ndarray
        The solution after relaxation as a 2D array of floats.
    ite : integer
        The number of iterations performed.
    conv : list
        The convergence history as a list of floats.
    """
    ny, nx = p0.shape
    p = p0.copy()
    conv = []  # convergence history
    diff = rtol + 1.0  # initial difference
    ite = 0  # iteration index
    while diff > rtol and ite < maxiter:
        pn = p.copy()
        # Update the solution at interior points.
        for j in range(1, ny - 1):
            for i in range(1, nx - 1):
                p[j, i] = 0.25 * (pn[j, i - 1] + pn[j, i + 1] +
                                  pn[j - 1, i] + pn[j + 1, i])
        # Apply 2nd-order Neumann condition (zero-gradient)
        # at the right boundary.
        for j in range(1, ny - 1):
            p[j, -1] = 0.25 * (2.0 * pn[j, -2] +
                               pn[j - 1, -1] + pn[j + 1, -1])
        # Compute the relative L2-norm of the difference.
        diff = numpy.sqrt(numpy.sum((p - pn)**2) / numpy.sum(pn**2))
        conv.append(diff)
        ite += 1
    return p, ite, conv

In [ ]:
# Compute solution using relaxed Jacobi method
p, ites, conv_jacobi = laplace_2d_jacobi(p0, maxiter=20000, rtol=1e-7)

print('Jacobi relaxation: {} iterations '.format(ites) + 
      'to reach relative diff of {}'.format(conv_jacobi[-1]))